In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.6/483.6 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.7 MB/s eta 0:00:00

In [ ]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pydoplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/머신러닝1/조별활동/와인품질분류/data

/content/drive/MyDrive/Colab Notebooks/머신러닝1/조별활동/와인품질분류/data


In [ ]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
clf = setup(data = train, target = 'quality', train_size = 0.9,  use_gpu = True, data_split_shuffle=False, normalize = True, session_id=42)

,Description,Value
0,Session id,6087
1,Target,quality
2,Target type,Multiclass
3,Target mapping,"3: 0, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5, 9: 6"
4,Original data shape,"(5497, 13)"
5,Transformed data shape,"(5497, 13)"
6,Transformed train set shape,"(3847, 13)"
7,Transformed test set shape,"(1650, 13)"
8,Numeric features,12
9,Preprocess,True


In [ ]:
best_5 = compare_models(sort = 'Accuracy', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.6558,0.3324,0.6558,0.6576,0.6391,0.4591,0.4657,0.2320
et,Extra Trees Classifier,0.6511,0.3368,0.6511,0.6496,0.6358,0.4535,0.4590,0.2710
xgboost,Extreme Gradient Boosting,0.6291,0.3236,0.6291,0.6218,0.6159,0.4245,0.4278,0.1030
lightgbm,Light Gradient Boosting Machine,0.6288,0.3213,0.6288,0.6242,0.6157,0.4244,0.4274,0.4140
gbc,Gradient Boosting Classifier,0.5792,0.3058,0.5792,0.5717,0.5608,0.3358,0.3430,0.7190
dt,Decision Tree Classifier,0.5635,0.2720,0.5635,0.5643,0.5623,0.3526,0.3534,0.0870
lda,Linear Discriminant Analysis,0.5272,0.2874,0.5272,0.5132,0.5059,0.2493,0.2558,0.0720
lr,Logistic Regression,0.5266,0.2800,0.5266,0.4842,0.4824,0.2246,0.2378,0.0680
ridge,Ridge Classifier,0.5207,0.0000,0.5207,0.4402,0.4542,0.2009,0.2195,0.0580
knn,K Neighbors Classifier,0.4627,0.2505,0.4627,0.4435,0.4467,0.1627,0.1644,0.0870


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

# 5개를 골라 블랜딩

In [ ]:
blended = blend_models(estimator_list = best_5, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6610,0.8292,0.6610,0.6826,0.6429,0.4664,0.4723
1,0.6195,0.8212,0.6195,0.6115,0.6096,0.4130,0.4149
2,0.6307,0.0000,0.6307,0.6300,0.6206,0.4249,0.4290
3,0.6528,0.8280,0.6528,0.6677,0.6362,0.4520,0.4587
4,0.6138,0.8170,0.6138,0.6097,0.6007,0.3957,0.4000
Mean,0.6356,0.6591,0.6356,0.6403,0.6220,0.4304,0.4350
Std,0.0185,0.3296,0.0185,0.0297,0.0158,0.0257,0.0269


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.6636,0.8448,0.6636,0.6632,0.6511,0.4740,0.4792


In [ ]:
final_model = finalize_model(blended)

In [ ]:
final_model

Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['fixed acidity',
                                             'volatile acidity', 'citric acid',
                                             'residual sugar', 'chlorides',
                                             'free sulfur dioxide',
                                             'total sulfur dioxide', 'density',
                                             'pH'...
                                                                          max_features=None,
                                                                          max_leaf_nodes=None,
                                                                          min_impurity_decrease=0.0,
                                                                          min_samples_leaf=1,
                                                                          min_samples_split=2,
                                                                          min_weight_fraction_leaf=0.0,
                                                                          n_estimators=100,
                                                                          n_iter_no_change=None,
                                                                          random_state=6087,
                                                                          subsample=1.0,
                                                                          tol=0.0001,
                                                                          validation_fraction=0.1,
                                                                          verbose=0,
                                                                          warm_start=False))],
                                  flatten_transform=True, n_jobs=-1,
                                  verbose=False, voting='soft',
                                  weights=None))],
         verbose=False)

In [ ]:
predictions = predict_model(final_model, data = test)

In [ ]:
predictions

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type,prediction_label,prediction_score
index,,,,,,,,,,,,,,
0,9.0,0.31,0.48,6.60,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,1,5,0.4172
1,13.3,0.43,0.58,1.90,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,0,5,0.4888
2,6.5,0.28,0.27,5.20,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,1,6,0.5667
3,7.2,0.15,0.39,1.80,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,1,5,0.6458
4,6.8,0.26,0.26,2.00,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,1,6,0.5651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,7.1,0.59,0.02,2.30,0.082,24.0,94.0,0.99744,3.55,0.53,9.7,0,6,0.5740
996,8.7,0.15,0.30,1.60,0.046,29.0,130.0,0.99420,3.22,0.38,9.8,1,6,0.5038
997,8.8,0.66,0.26,1.70,0.074,4.0,23.0,0.99710,3.15,0.74,9.2,0,5,0.7074


In [ ]:
pred = predictions['prediction_label']
pred

index
0      5
1      5
2      6
3      5
4      6
      ..
995    6
996    6
997    5
998    6
999    6
Name: prediction_label, Length: 1000, dtype: int64

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['quality'] = pred

TypeError: ignored

In [ ]:
submission.to_csv('sub.csv',index=False)

In [ ]:
0.708 50등